In [4]:
# Step 1: Import the package
try:
    import onnx
    from onnx import helper, TensorProto, checker
    import numpy as np
    import os
    print("Step 1: Importing onnx, numpy, and os passed.")
except ImportError:
    print("Step 1 failed: 'onnx', 'numpy', or 'os' is not installed.")
    sys.exit(1)
except Exception as e:
    print(f"Step 1 failed with an unexpected error: {str(e)}")
    sys.exit(1)

# Step 2: Create a simple ONNX model
try:
    # Define the model input (1x3x224x224 tensor)
    input_tensor = helper.make_tensor_value_info('input', TensorProto.FLOAT, [1, 3, 224, 224])

    # Define the model output (1x3x224x224 tensor)
    output_tensor = helper.make_tensor_value_info('output', TensorProto.FLOAT, [1, 3, 224, 224])

    # Create a simple node that performs an element-wise addition of the input with itself
    add_node = helper.make_node(
        'Add',  # Operator name
        ['input', 'input'],  # Inputs
        ['output']  # Outputs
    )

    # Create the graph
    graph_def = helper.make_graph(
        [add_node],
        'simple-addition-graph',  # Graph name
        [input_tensor],  # Inputs
        [output_tensor]  # Outputs
    )

    # Create the model
    model_def = helper.make_model(graph_def, producer_name='onnx-example')
    
    print("Step 2: Creating a simple ONNX model passed.")
except Exception as e:
    print(f"Step 2 failed: {str(e)}")
    sys.exit(1)

# Step 3: Validate the created ONNX model
try:
    checker.check_model(model_def)
    print("Step 3: Model validation passed.")
except onnx.checker.ValidationError as e:
    print(f"Step 3 failed: Model validation error - {str(e)}")
    sys.exit(1)
except Exception as e:
    print(f"Step 3 failed: {str(e)}")
    sys.exit(1)

# Step 4: Print model information
try:
    model_info = onnx.helper.printable_graph(model_def.graph)
    print("Step 4: Model information:")
    print(model_info)
    print("Step 4: Printing model information passed.")
except Exception as e:
    print(f"Step 4 failed: {str(e)}")
    sys.exit(1)

# Step 5: Perform inference using the ONNX model with ONNX Runtime
try:
    import onnxruntime as ort
    
    # Save the model to a file to use with ONNX Runtime
    onnx.save(model_def, 'simple_model.onnx')
    
    # Create an ONNX Runtime session
    session = ort.InferenceSession('simple_model.onnx')
    
    # Generate a dummy input matching the model's input shape
    dummy_input = np.random.randn(1, 3, 224, 224).astype(np.float32)
    
    # Perform inference
    input_name = session.get_inputs()[0].name
    output_name = session.get_outputs()[0].name
    result = session.run([output_name], {input_name: dummy_input})
    
    assert result is not None, "Inference failed, no output received."
    assert np.allclose(result, dummy_input + dummy_input), "Inference result is not as expected."
    print("Step 5: Performing inference using ONNX Runtime passed.")
except ImportError:
    print("Step 5 failed: 'onnxruntime' is not installed.")
    sys.exit(1)
except Exception as e:
    print(f"Step 5 failed: {str(e)}")
    sys.exit(1)

# Step 6: Save the ONNX model to a new file
try:
    onnx.save(model_def, 'saved_simple_model.onnx')
    assert os.path.exists('saved_simple_model.onnx'), "Model save failed, file not created."
    print("Step 6: Saving the ONNX model passed.")
except Exception as e:
    print(f"Step 6 failed: {str(e)}")
    sys.exit(1)

# Final Confirmation
print("All extensive tests for the 'onnx' package completed successfully.")


Step 1: Importing onnx, numpy, and os passed.
Step 2: Creating a simple ONNX model passed.
Step 3: Model validation passed.
Step 4: Model information:
graph simple-addition-graph (
  %input[FLOAT, 1x3x224x224]
) {
  %output = Add(%input, %input)
  return %output
}
Step 4: Printing model information passed.
Step 5: Performing inference using ONNX Runtime passed.
Step 6: Saving the ONNX model passed.
All extensive tests for the 'onnx' package completed successfully.
